<a href="https://colab.research.google.com/github/alexeiveselov92/Yandex-Data-Analysis/blob/master/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD_%D1%8F%D0%BD%D0%B4%D0%B5%D0%BA%D1%81_%D0%B3%D0%B5%D0%BE%D0%BA%D0%BE%D0%B4%D0%B5%D1%80_%2B_%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA%D0%B0%D1%80%D1%82%D1%8B_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import numpy as np
#подключение к яндекс геокодеру
token = '8a8c140c-039b-44d3-b287-373471d8c2a1'


In [0]:
#добавляем список адресов
list_of_objects = [
                   'СПБ, ЖК Континенты',
                   'СПБ, ЖК Бомонд В.О.',
                   'СПБ, ЖК YOGA',
                   'СПБ, В.О, 6-линия д. 59',
                   'СПБ, ЖК Приморский квартал',
                   'СПБ, ЖК Орловский парк',
                   'СПБ, ЖК Белорусский квартал',
                   'СПБ, ЖК Заневский штиль',
                   'СПБ, ЖК 4YOU',
                   'СПБ, ЖК Калейдоскоп',
                   'СПБ, ЖК Старт',
                   'СПБ, ЖК Шуваловский',
                   'СПБ, ЖК Лесная Life',
                   'СПБ, ЖК Yoga',
                   'СПБ, Veda Village',
                   'СПБ, ЖК Море океан',
                   'СПБ, ЖК Балтийская жемчужина',
                   'СПБ, Студенческая улица, дом 24, литера А',
                   'СПБ, ЖК Ультрасити',
                   'СПБ, ЖК Граффити',
                   'СПБ, ЖК Территория'
]

#добавляем список менеджеров
list_of_managers = [
                   'Голеров',
                   'Мацкевич',
                   'Ванюшичев',
                   'Голеров',
                   'Ванюшичев',
                   'Голеров',
                   'Ванюшичев',
                   'Ванюшичев',
                   'Мацкевич',
                   'Голеров',
                   'Голеров',
                   'Мацкевич',
                   'Мацкевич',
                   'Ванюшичев',
                   'Ванюшичев',
                   'Мацкевич',
                   'Ванюшичев',
                   'Голеров',
                   'Мацкевич',
                   'Голеров',
                   'Ванюшичев'
]

In [0]:
#получаем соответствующий адресам список координат
url = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey=' + token + '&geocode='

coordinates_list = []

for address in list_of_objects:
  link = url + address
  response = requests.get(link).json()
  try:
    coordinates = response['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point'].get('pos')
    coordinates = (float(coordinates.split()[1]), float(coordinates.split()[0]))
    coordinates_list.append(coordinates)
  except:
    coordinates_list.append('')


In [0]:
#объединяем списки адресов и менеджеров со списком координат в один датафрейм
data = pd.DataFrame({'address':list_of_objects, 'coordinates':coordinates_list, 'managers':list_of_managers})

In [5]:
#создаем карту через библиотеку folium и добавляем на неё наши объекты

import folium
from folium.plugins import MarkerCluster

#создание карты
map = folium.Map(location=[59.985512, 30.378633], zoom_start = 10)

#подготовка датафрейма к передаче данных в библиотеку folium
address = data['address']
coordinates = data['coordinates']
managers =data['managers']

#напишем функцию выбора цвета в зависимости от менеджера
def color_change(manager):
  if manager == 'Ванюшичев':
    return ('green')
  elif manager == 'Голеров':
    return ('red')
  elif manager == 'Мацкевич':
    return ('blue')

#создадим кластер
marker_cluster = MarkerCluster().add_to(map)

#добавляем метки на карту циклом
for coordinates, address, managers in zip(coordinates, address, managers):
    folium.CircleMarker(location=coordinates, radius = 7, 
                        popup=str(address + ', ' + managers), 
                        fill_color = color_change(managers),
                        color = color_change(managers), 
                        fill_opacity = 0.7).add_to(marker_cluster)
                        

map.save("map1.html")
map